##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Guía completa de agrupación de pesos

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

Le damos la bienvenida la guía completa de *agrupación de pesos*, que forma parte del kit de herramientas de optimización de modelos de TensorFlow.

En esta página, se documentan varios casos de uso y se muestra cómo usar la API para cada uno. Una vez que sepa qué API necesita, busque los parámetros y los detalles de bajo nivel en los [documentos de API](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/clustering):

- Si quiere ver los beneficios de la agrupación de pesos y lo que se admite, consulte la [descripción general](https://www.tensorflow.org/model_optimization/guide/clustering).
- Para ver un ejemplo único de principio a fin, consulte el [ejemplo de agrupación de pesos](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_example).

En esta guía, se tratan los siguientes casos de uso:

- Definir un modelo agrupado.
- Establecer punto de verificación y deserializar un modelo agrupado.
- Mejorar la precisión del modelo agrupado.
- Solo para la implementación, debe tomar medidas para ver los beneficios de la compresión.


## Preparación


In [ ]:
! pip install -q tensorflow-model-optimization

import tensorflow as tf
import numpy as np
import tempfile
import os
import tensorflow_model_optimization as tfmot

input_dim = 20
output_dim = 20
x_train = np.random.randn(1, input_dim).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=output_dim)

def setup_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(input_dim, input_shape=[input_dim]),
      tf.keras.layers.Flatten()
  ])
  return model

def train_model(model):
  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
  )
  model.summary()
  model.fit(x_train, y_train)
  return model

def save_model_weights(model):
  _, pretrained_weights = tempfile.mkstemp('.h5')
  model.save_weights(pretrained_weights)
  return pretrained_weights

def setup_pretrained_weights():
  model= setup_model()
  model = train_model(model)
  pretrained_weights = save_model_weights(model)
  return pretrained_weights

def setup_pretrained_model():
  model = setup_model()
  pretrained_weights = setup_pretrained_weights()
  model.load_weights(pretrained_weights)
  return model

def save_model_file(model):
  _, keras_file = tempfile.mkstemp('.h5') 
  model.save(keras_file, include_optimizer=False)
  return keras_file

def get_gzipped_model_size(model):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  keras_file = save_model_file(model)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)
  return os.path.getsize(zipped_file)

setup_model()
pretrained_weights = setup_pretrained_weights()

## Definir un modelo agrupado


### Agrupar un modelo completo (secuencial y funcional)

**Consejos** para una mejor precisión del modelo:

- Debe pasar un modelo preentrenado con una precisión aceptable para esta API. Entrenar modelos desde cero con agrupamiento da como resultado una precisión inferior.
- En algunos casos, agrupar determinadas capas tiene un efecto perjudicial en la precisión del modelo. Marque "Cluster some layers" (Agrupar algunas capas) para ver cómo omitir la agrupación de las capas que más afectan la precisión.

Para agrupar todas las capas, aplique `tfmot.clustering.keras.cluster_weights` al modelo.


In [ ]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 3,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

model = setup_model()
model.load_weights(pretrained_weights)

clustered_model = cluster_weights(model, **clustering_params)

clustered_model.summary()

### Agrupar algunas capas (modelos secuenciales y funcionales)


**Consejos** para una mejor precisión del modelo:

- Debe pasar un modelo preentrenado con una precisión aceptable para esta API. Entrenar modelos desde cero con agrupamiento da como resultado una precisión inferior.
- Agrupe las capas posteriores con parámetros más redundantes (por ejemplo, `tf.keras.layers.Dense`, `tf.keras.layers.Conv2D`), a diferencia de las primeras capas.
- Congele las primeras capas antes de las capas agrupadas durante el ajuste. Trate el número de capas congeladas como un hiperparámetro. Empíricamente, congelar la mayoría de las primeras capas es ideal para la API de agrupación actual.
- Evite agrupar capas críticas (por ejemplo, mecanismo de atención).

**Más**: en los documentos de la API `tfmot.clustering.keras.cluster_weights` se pueden encontrar detalles sobre cómo variar la configuración de la agrupación por capas.

In [ ]:
# Create a base model
base_model = setup_model()
base_model.load_weights(pretrained_weights)

# Helper function uses `cluster_weights` to make only 
# the Dense layers train with clustering
def apply_clustering_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return cluster_weights(layer, **clustering_params)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_clustering_to_dense` 
# to the layers of the model.
clustered_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_clustering_to_dense,
)

clustered_model.summary()

### Agrupar capas convolucionales por canal

El modelo agrupado podría pasarse por más optimizaciones, como una [cuantización posentrenamiento](https://www.tensorflow.org/lite/performance/post_training_quantization). Si se realiza la cuantización por canal, entonces el modelo también debe agruparse por canal. Esto aumenta la precisión del modelo agrupado y cuantizado.

**Nota:** solo las capas Conv2D están agrupadas por canal

Para agrupar por canal, se debe establecer el parámetro `cluster_per_channel` como `True`. Podría configurarse para algunas capas o para todo el modelo.

**Consejos:**

- Si se va a cuantificar aún más un modelo, puede considerar usar la [técnica con reconocimiento de cuantización (QAT) de preservación de clústeres](https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization).

- Se podría podar el modelo antes de aplicar la agrupación por canal. Con el parámetro `preserve_sparsity` establecido en `True`, la dispersión se conserva durante la agrupación por canal. Tenga en cuenta que en este caso se debe usar la [técnica con reconocimiento de cuantización (QAT) de preservación de clústeres y dispersión](https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization).

### Agrupar la capa Keras personalizada o especificar qué pesos de capa agrupar

`tfmot.clustering.keras.ClusterableLayer` sirve para dos casos de uso:

1. Agrupar cualquier capa que no sea compatible de forma nativa, incluida una capa Keras personalizada.
2. Especificar qué pesos de una capa admitida se agruparán.

Por ejemplo, la API por defecto agrupa solo el núcleo de la capa `Dense`. En el siguiente ejemplo, se muestra cómo modificarlo para agrupar también el sesgo. Tenga en cuenta que al derivar de la capa Keras, se debe anular la función `get_clusterable_weights`, donde se especifica el nombre de la variable entrenable que se agrupará y la variable entrenable en sí. Por ejemplo, si devuelve una lista vacía [], no se podrá agrupar ningún peso.

**Error común:** agrupar el sesgo suele perjudicar demasiado la precisión del modelo.

In [ ]:
class MyDenseLayer(tf.keras.layers.Dense, tfmot.clustering.keras.ClusterableLayer):

  def get_clusterable_weights(self):
   # Cluster kernel and bias. This is just an example, clustering
   # bias usually hurts model accuracy.
   return [('kernel', self.kernel), ('bias', self.bias)]

# Use `cluster_weights` to make the `MyDenseLayer` layer train with clustering as usual.
model_for_clustering = tf.keras.Sequential([
  tfmot.clustering.keras.cluster_weights(MyDenseLayer(20, input_shape=[input_dim]), **clustering_params),
  tf.keras.layers.Flatten()
])

model_for_clustering.summary()

También puede usar `tfmot.clustering.keras.ClusterableLayer` para agrupar una capa personalizada de Keras. Para hacer esto, debe ampliar `tf.keras.Layer` como de costumbre e implementar las funciones `__init__`, `call` y `build`, pero también se debe amploar la clase `clusterable_layer.ClusterableLayer` e implementar:

1. `get_clusterable_weights`, donde se especifica el núcleo de peso que se agrupará, como se muestra anteriormente.
2. `get_clusterable_algorithm`, donde se especifica el algoritmo de agrupamiento para el tensor de peso. Se hace así porque se debe especificar cómo se configuran los pesos de las capas personalizadas para la agrupación. La clase de algoritmo de agrupamiento que se devuelve debe derivarse de la clase `clustering_algorithm.ClusteringAlgorithm` y la se debe sobreescribir la función `get_pulling_indices`. Puede encontrar un ejemplo de esta función, que admite pesos de rangos unidimensionales, bidimensionales y tridimencionales [aquí](https://github.com/tensorflow/model-optimization/blob/18e87d262e536c9a742aef700880e71b47a7f768/tensorflow_model_optimization/python/core/clustering/keras/clustering_algorithm.py#L62).

Puede encontrar un ejemplo de este caso de uso [aquí](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/clustering/keras/mnist_clusterable_layer_test.py).

## Establecer punto de verificación y deserializar un modelo agrupado

**Su caso de uso:** este código solo es necesario para el formato del modelo HDF5 (no para pesos HDF5 u otros formatos).

In [ ]:
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights)
clustered_model = cluster_weights(base_model, **clustering_params)

# Save or checkpoint the model.
_, keras_model_file = tempfile.mkstemp('.h5')
clustered_model.save(keras_model_file, include_optimizer=True)

# `cluster_scope` is needed for deserializing HDF5 models.
with tfmot.clustering.keras.cluster_scope():
  loaded_model = tf.keras.models.load_model(keras_model_file)

loaded_model.summary()

## Mejorar la precisión del modelo agrupado

Para su caso de uso específico, tenga en cuenta estos consejos:

- La inicialización del centroide juega un papel clave en la precisión del modelo optimizado final. En general, la inicialización kmeans++ supera a la inicialización lineal, de densidad y aleatoria. Cuando no se usa kmeans++, la inicialización lineal tiende a superar la densidad y la inicialización aleatoria, ya que no suele omitir pesos grandes. Sin embargo, se ha observado que la inicialización de la densidad proporciona una mayor precisión en el caso de usar muy pocos grupos en pesos con distribuciones bimodales.

- Establezca una tasa de aprendizaje que sea menor que la del entrenamiento al ajustar el modelo agrupado.

- Para obtener ideas generales para mejorar la precisión del modelo, busque sugerencias para sus casos de uso en "Definir un modelo agrupado".

## Implementación

### Exportar modelo con compresión de tamaño

**Error común**: tanto <code>strip_clustering</code> como la aplicación de un algoritmo de compresión estándar (por ejemplo, mediante gzip) son necesarios para ver los beneficios de la compresión de la agrupación.

In [ ]:
model = setup_model()
clustered_model = cluster_weights(model, **clustering_params)

clustered_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

clustered_model.fit(
    x_train,
    y_train
)

final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

print("final model")
final_model.summary()

print("\n")
print("Size of gzipped clustered model without stripping: %.2f bytes" 
      % (get_gzipped_model_size(clustered_model)))
print("Size of gzipped clustered model with stripping: %.2f bytes" 
      % (get_gzipped_model_size(final_model)))